# Prepare different HI surveys for HiPS building
In this notebook, I take the various HI surveys that I could lay hands on and make RGB HiPS for each of them individually and one for all of them together. The general idea for the RGB HiPS, regardless of the data included, is the following:
 1. Look at the recession velocity of each observation, either centroid of spectrum from cube or median in mom1 map
 2. sort, within the sample of maps or cubes to be hipsified, the data in three percentiles, according to their recession velocity, i.e. one group with the highest third of recession velocities (red), one group with the intermediate third of recession velocities (green) and one group with the lowest third of recession velocities (blue). 
 3. For moment 1 maps prepare the data such that 0km/s is at the recession velocity (distance will be encoded in the RGB channels), make sure there are only 2 dimensions.
 4. For moment 0 maps prepare the data such that the unit is Msun/pc2, make sure there are only 2 dimensions. With moment0 maps we can either have one grey scale HiPS, then distance information is lost. Or we can create an RGB image, again in three percentiles of recession velocities. 
 5. For cubes prepare the data such that the velocity plane of the recession velocity is at 0km/s (again distance will be encoded in the RGB channels), make sure there are only 3 dimensions. BUT, different cubes from different surveys will have different velocity channel numbers and widths --> adjust, i.e. in the case of one HiPS for all surveys degrade to the largest channel width, also check for data within one survey!
 6. Do the data sorting in folders. 

In [1]:
import glob
import shutil
import os 

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from specutils import Spectrum1D
import specutils.analysis as spec_ana

In [2]:
def create_wave_array(cube):
    wave = np.array([(cube[0].header['CRVAL3'] + j * cube[0].header['CDELT3'])
                for j in range(cube[0].header['NAXIS3'])])
    return wave

In [3]:
def get_rec_velo_cube(cube):
    wave = create_wave_array(cube)
    rec_velo = np.median(wave)
    return rec_velo

In [4]:
def get_sorting(cube_list):
    ''' This function takes a glob list of cubes, calculates 
        for every cube the recession velocity and then creates
        a table with the ID of the cube, the survey name = name
        of the folder within which the data of the survey is 
        located, and the recession velocity. This table is saved 
        and for within one survey the percentile of recession
        velocity is also noted: 1 --> lowest velos = blue, 
        2 --> intermediate velocities = green, 3 --> highes velos =
        red
    '''
    rec_velo_list = []
    id_list = []
    for m in cube_list:
        cube = fits.open(m)
        rec_velo_list.append(get_rec_velo_cube(cube))
        cube.close()
        name = m.split('_cube.fits')[0].split('/')[-1]
        id_list.append(name)
    tab_out = Table([id_list, rec_velo_list], names=('ID', 'Vrec'))
    tab_out['Velo Third'] = -99
    tab_out['Colour'] = 'colour'
    ind_lower = tab_out['Vrec'] < np.percentile(tab_out['Vrec'], 33.3)
    print(len(tab_out[ind_lower]))
    tab_out['Velo Third'][ind_lower] = 1 
    tab_out['Colour'][ind_lower] = 'Blue'
    ind_med = (tab_out['Vrec'] >= np.percentile(tab_out['Vrec'], 33.3)) & \
        (tab_out['Vrec'] < np.percentile(tab_out['Vrec'], 66.6))
    print(len(tab_out[ind_med]))
    tab_out['Velo Third'][ind_med] = 2
    tab_out['Colour'][ind_med] = 'Green'
    ind_high = tab_out['Vrec'] >= np.percentile(tab_out['Vrec'], 66.6)
    print(len(tab_out[ind_high]))
    tab_out['Velo Third'][ind_high] = 3 
    tab_out['Colour'][ind_high] = 'Red'
    return tab_out

In [5]:
def get_sorting_mom1(mom1_list):
    ''' This function takes a glob list of cubes, calculates 
        for every cube the recession velocity and then creates
        a table with the ID of the cube, the survey name = name
        of the folder within which the data of the survey is 
        located, and the recession velocity. This table is saved 
        and for within one survey the percentile of recession
        velocity is also noted: 1 --> lowest velos = blue, 
        2 --> intermediate velocities = green, 3 --> highes velos =
        red
    '''
    rec_velo_list = []
    id_list = []
    for m in mom1_list:
        mom1 = fits.open(m)
        rec_velo_list.append(get_rec_velo_cube(cube))
        mom1.close()
        name = m.split('_mom1.fits')[0].split('/')[-1]
        id_list.append(name)
    tab_out = Table([id_list, rec_velo_list], names=('ID', 'Vrec'))
    tab_out['Velo Third'] = -99
    tab_out['Colour'] = 'colour'
    ind_lower = tab_out['Vrec'] < np.percentile(tab_out['Vrec'], 33.3)
    print(len(tab_out[ind_lower]))
    tab_out['Velo Third'][ind_lower] = 1 
    tab_out['Colour'][ind_lower] = 'Blue'
    ind_med = (tab_out['Vrec'] >= np.percentile(tab_out['Vrec'], 33.3)) & \
        (tab_out['Vrec'] < np.percentile(tab_out['Vrec'], 66.6))
    print(len(tab_out[ind_med]))
    tab_out['Velo Third'][ind_med] = 2
    tab_out['Colour'][ind_med] = 'Green'
    ind_high = tab_out['Vrec'] >= np.percentile(tab_out['Vrec'], 66.6)
    print(len(tab_out[ind_high]))
    tab_out['Velo Third'][ind_high] = 3 
    tab_out['Colour'][ind_high] = 'Red'
    return tab_out

In [6]:
def get_beam_params_whisp(hdr):
    hdr['BMAJ'] = hdr['BMMAJ'] / 3600.
    hdr['BMIN'] = hdr['BMMIN'] / 3600.
    hdr['BPA'] = hdr['BMPA']
    return hdr

In [7]:
def get_beam_params_vla(hdr):
    bmaj, bmin, bpa = -99.9, -99.9, -99.9
    for line in hdr['HISTORY']:
        if line[: 17] == 'AIPS   CLEAN BMAJ':
            x = np.array(line.split(' '))
            x = x[x != '']
            bmaj = x[3]
            bmin = x[5]
            bpa = x[7]
    hdr['BMAJ'] = float(bmaj)
    hdr['BMIN'] = float(bmin)
    hdr['BPA'] = float(bpa)
    return hdr

In [8]:
def update_coord_sys(hdr):
    c = SkyCoord(ra=hdr['CRVAL1'], dec=hdr['CRVAL2'], 
                 unit=u.deg, frame='fk4')
    cfk5 = c.transform_to('fk5')
    hdr['EPOCH'] = 2000.0
    hdr['CRVAL1'] = cfk5.ra.value
    hdr['CRVAL2'] = cfk5.dec.value
    return hdr

In [9]:
def update_header(ima):
    '''This function extracts the beam parameters from comment
        or history cards if they are not available in the 
        standard 'BMAJ', 'BMIN', 'BPA' header keywords. Then the
        beam parameters are written to the standard header keywords. 
    '''
    if 'BMMAJ' in ima[0].header:
        ima[0].header = get_beam_params_whisp(ima[0].header)
    if ('BMMAJ' not in ima[0].header) & ('BMAJ' not in ima[0].header):
        ima[0].header = get_beam_params_vla(ima[0].header)
    if ('EPOCH' in ima[0].header):
        if (ima[0].header['EPOCH'] == 1950.):
            ima[0].header = update_coord_sys(ima[0].header)
    if ima[0].header['BMAJ'] == -99.9:
        print(ima.info())
        return 0
    else:
        return ima

In [10]:
def calc_column_dens(mom0):
    mom0[0].data = 1.823e18 * ((606. * mom0[0].data * 1000.) / 
                               (mom0[0].header['BMAJ'] * 3600. * 
                                mom0[0].header['BMIN'] * 3600.)) #cm^-2
    mom0[0].data = mom0[0].data * 8.044e-21 #M_sun / pc^2
    return mom0

In [11]:
def red_dimension_3_2(mom):
    mom[0].header['NAXIS'] = 2
    del_hdr_keywords = ['CDELT3', 'CRPIX3', 'CRVAL3', 'CTYPE3', 
                        'CUNIT3', 'NAXIS3', 'DRVAL3', 'DTYPE3', 
                        'DUNIT3', 'CDELT4', 'CRVAL4', 'CTYPE4',
                        'CUNIT4', 'DRVAL4', 'DTYPE4', 'DUNIT4']
    for keyword in del_hdr_keywords:
        if keyword in mom[0].header:
            del mom[0].header[keyword]
    try:
        mom[0].data = mom[0].data.reshape((mom[0].data.shape[1], 
                                           mom[0].data.shape[2]))
    except IndexError:
        print('Data has already been reshaped')
    return mom

In [12]:
def red_dimension_4_3(cube):
    cube[0].header['NAXIS'] = 3
    del_hdr_keywords = ['CUNIT4', 'NAXIS4',
                        'CDELT4', 'CRPIX4', 'CRVAL4', 'CTYPE4']
    for keyword in del_hdr_keywords:
        if keyword in cube[0].header:
            del cube[0].header[keyword]
    try:
        cube[0].data = cube[0].data.reshape((cube[0].data.shape[1], 
                                             cube[0].data.shape[2], 
                                             cube[0].data.shape[3]))
    except IndexError:
        print('Data has already been reshaped')
    return cube

In [13]:
def prepare_mom0(mom0_name, color, parent_folder):
    '''This function makes sure that there are only two dimensions in 
        the moment 0 map, adds the beam parameters with the right 
        header keywords if neccessary, converts the data in the moment 
        0 maps to units of Msun/pc2 and saves the prepared 
        moment 0 maps to the right folder (depending on 
        the recession velocity.)
        parent_folder = e.g. '/data/Survey/VIVA/Mom0/'
    '''
    print('Moment 0 ...')
    mom0 = fits.open(parent_folder + mom0_name)
    if mom0[0].header['NAXIS'] == 3:
        mom0 = red_dimension_3_2(mom0)
    mom0 = update_header(mom0)
    if mom0 == 0:
        return 0
#     mom0 = calc_column_dens(mom0)
    mom0.writeto('{}{}/{}'.format(parent_folder, color, mom0_name))
    return 1

In [14]:
def prepare_mom1(mom1_name, color, parent_folder):
    '''This function makes sure that there are only two dimensions in 
        the moment 1 map, subtracts the recession velocity from 
        the map data and saves the prepared moment 1 maps to the 
        right folder (depending on the recession velocity.)
    '''
    print('Moment 1 ...')
    mom1 = fits.open(parent_folder + mom1_name)
    if mom1[0].header['NAXIS'] == 3:
        mom1 = red_dimension_3_2(mom1)
    mom1 = update_header(mom1)
    if mom1 == 0:
        return 0
#     ind = (mom1[0].data > 0.0) & (mom1[0].data < 1.0e8)
#     mom1[0].data = mom1[0].data - np.median(mom1[0].data[ind])
    mom1.writeto('{}{}/{}'.format(parent_folder, color, mom1_name))
    return 1

In [15]:
def prepare_cubes(cube_name, color, parent_folder):
    '''This function makes sure that there are only three dimensions
        in the cube, adjusts the wcs velocity axis such that the 
        recession velocity is at 0 and that we are within the optical 
        convention. Then it saves the cube to the right folder (depending
        on the recession velocity).
    '''
    print('Cube ...')
    cube = fits.open(parent_folder + cube_name)
    if cube[0].header['NAXIS'] == 4:
        cube = red_dimension_4_3(cube)
    cube = update_header(cube) 
    if cube == 0:
        return 0
    cube.writeto('{}{}/{}'.format(parent_folder, color, cube_name))
    return 1

In [16]:
def sort_the_data(base_folder, tab):
    dirs = ['Mom0/Red', 'Mom0/Green', 'Mom0/Blue', 
            'Mom1/Red', 'Mom1/Green', 'Mom1/Blue', 
            'Cubes/Red', 'Cubes/Green', 'Cubes/Blue'
           ]
    for d in dirs:
        if os.path.isdir('{}{}'.format(base_folder, d)): 
            shutil.rmtree('{}{}'.format(base_folder, d))
        os.mkdir('{}{}'.format(base_folder, d))
    for gal in tab['ID']:
        print(gal)
        ind = tab['ID'] == gal
        if os.path.isfile('{}Mom0/{}_mom0.fits'.format(base_folder, gal)):
           prepare_mom0('{}_mom0.fits'.format(gal), 
                        tab['Colour'][ind][0], '{}Mom0/'.format(base_folder))
        if os.path.isfile('{}Mom1/{}_mom1.fits'.format(base_folder, gal)):
            prepare_mom1('{}_mom1.fits'.format(gal), 
                         tab['Colour'][ind][0], '{}Mom1/'.format(base_folder))
        out = prepare_cubes('{}_cube.fits'.format(gal), 
                            tab['Colour'][ind][0], '{}Cubes/'.format(base_folder))
        print(gal, out)

## Atlas3D

In [17]:
mom1_list = glob.glob('/data/Surveys/Atlas3D/Cubes/*.fits')
atlas_tab = get_sorting(mom1_list)
atlas_tab.write('ATLAS3D.csv', overwrite=True)
sort_the_data('/data/Surveys/Atlas3D/', atlas_tab)
atlas_tab

43
43
43
NGC3489
Moment 0 ...
Moment 1 ...
Cube ...
NGC3489 1
NGC2549
Moment 0 ...
Moment 1 ...
Cube ...
NGC2549 1
NGC3796
Moment 0 ...
Moment 1 ...
Cube ...
NGC3796 1
NGC3377
Moment 0 ...
Moment 1 ...
Cube ...
NGC3377 1
NGC2950
Moment 0 ...
Moment 1 ...
Cube ...
NGC2950 1
NGC5500
Moment 0 ...
Moment 1 ...
Cube ...
NGC5500 1
NGC6278
Moment 0 ...
Moment 1 ...
Cube ...
NGC6278 1
NGC4026
Moment 0 ...
Moment 1 ...
Cube ...
NGC4026 1
NGC2778
Moment 0 ...
Moment 1 ...
Cube ...
NGC2778 1
NGC2592
Moment 0 ...
Moment 1 ...
Cube ...
NGC2592 1
NGC5475
Moment 0 ...
Moment 1 ...
Cube ...
NGC5475 1
NGC3530
Moment 0 ...
Moment 1 ...
Cube ...
NGC3530 1
NGC2764
Moment 0 ...
Moment 1 ...
Cube ...
NGC2764 1
PGC051753
Moment 0 ...
Moment 1 ...
Cube ...
PGC051753 1
IC3631
Moment 0 ...
Moment 1 ...
Cube ...
IC3631 1
NGC3073
Moment 0 ...
Moment 1 ...
Cube ...
NGC3073 1
NGC4552
Moment 0 ...
Moment 1 ...
Cube ...
NGC4552 1
NGC4168
Moment 0 ...
Moment 1 ...
Cube ...
NGC4168 1
NGC4143
Moment 0 ...
Moment 1 ...
C

ID,Vrec,Velo Third,Colour
str9,float64,int64,str6
NGC3489,-806518.9326674751,1,Blue
NGC2549,1498628.2329571699,1,Blue
NGC3796,1174102.61543104,1,Blue
NGC3377,1325042.722005365,1,Blue
NGC2950,1646504.9259039098,2,Green
NGC5500,1964534.55436536,2,Green
NGC6278,2498251.5379113005,3,Red
NGC4026,1444470.207341165,1,Blue
NGC2778,1995946.5010796848,2,Green


## BLUEDISK

In [18]:
mom1_list = glob.glob('/data/Surveys/BLUEDISK/Cubes/*.fits')
bluedisk_tab = get_sorting(mom1_list)
bluedisk_tab.write('BLUEDISK.csv', overwrite=True)
sort_the_data('/data/Surveys/BLUEDISK/', bluedisk_tab)
bluedisk_tab

16
16
17
j164714-401442
Moment 0 ...
Moment 1 ...
Cube ...
j164714-401442 1
j072745-421050
Moment 0 ...
Moment 1 ...
Cube ...
j072745-421050 1
j082846-403957
Moment 0 ...
Moment 1 ...
Cube ...
j082846-403957 1
j091458-512139
Moment 0 ...
Moment 1 ...
Cube ...
j091458-512139 1
j091645-454844
Moment 0 ...
Moment 1 ...
Cube ...
j091645-454844 1
j130713-580806
Moment 0 ...
Moment 1 ...
Cube ...
j130713-580806 1
j091424-404640
Moment 0 ...
Moment 1 ...
Cube ...
j091424-404640 1
j083833-304755
Moment 0 ...
Moment 1 ...
Cube ...
j083833-304755 1
j160734-362902
Moment 0 ...
Moment 1 ...
Cube ...
j160734-362902 1
j111415-340915
Moment 0 ...
Moment 1 ...
Cube ...
j111415-340915 1
j114657-504207
Moment 0 ...
Moment 1 ...
Cube ...
j114657-504207 1
j134100-422553
Moment 0 ...
Moment 1 ...
Cube ...
j134100-422553 1
j101610-582537
Moment 0 ...
Moment 1 ...
Cube ...
j101610-582537 1
j172837-570842
Moment 0 ...
Moment 1 ...
Cube ...
j172837-570842 1
j172613-620858
Moment 0 ...
Moment 1 ...
Cube ...
j17

ID,Vrec,Velo Third,Colour
str14,float64,int64,str6
j164714-401442,1375357087.2741013,1,Blue
j072745-421050,1384116111.7598395,3,Red
j082846-403957,1382231345.583973,3,Red
j091458-512139,1378093980.4725113,2,Green
j091645-454844,1379822713.8604417,2,Green
j130713-580806,1377952741.9519749,2,Green
j091424-404640,1377500099.0403013,1,Blue
j083833-304755,1380618201.7621331,3,Red
j160734-362902,1375028870.8022413,1,Blue


## HALOGAS

In [19]:
mom1_list = glob.glob('/data/Surveys/HALOGAS/Cubes/*.fits')
halogas_tab = get_sorting(mom1_list)
halogas_tab.write('HALOGAS.csv', overwrite=True)
sort_the_data('/data/Surveys/HALOGAS/', halogas_tab)
halogas_tab

8
8
8
NGC0672
Moment 0 ...
Moment 1 ...
Cube ...
NGC0672 1
NGC4244
Moment 0 ...
Moment 1 ...
Cube ...
NGC4244 1
NGC4448
Moment 0 ...
Moment 1 ...
Cube ...
NGC4448 1
NGC0891
Moment 0 ...
Moment 1 ...
Cube ...
NGC0891 1
NGC2541
Moment 0 ...
Moment 1 ...
Cube ...
NGC2541 1
NGC5585
Moment 0 ...
Moment 1 ...
Cube ...
NGC5585 1
NGC0949
Moment 0 ...
Moment 1 ...
Cube ...
NGC0949 1
NGC5229
Moment 0 ...
Moment 1 ...
Cube ...
NGC5229 1
NGC0925
Moment 0 ...
Moment 1 ...
Cube ...
NGC0925 1
NGC4414
Moment 0 ...
Moment 1 ...
Cube ...
NGC4414 1
NGC5023
Moment 0 ...
Moment 1 ...
Cube ...
NGC5023 1
NGC4565
Moment 0 ...
Moment 1 ...
Cube ...
NGC4565 1
UGC7774
Moment 0 ...
Moment 1 ...
Cube ...
UGC7774 1
NGC5055
Moment 0 ...
Moment 1 ...
Cube ...
NGC5055 1
UGC2082
Moment 0 ...
Moment 1 ...
Cube ...
UGC2082 1
NGC3198
Moment 0 ...
Moment 1 ...
Cube ...
NGC3198 1
UGC4278
Moment 0 ...
Moment 1 ...
Cube ...
UGC4278 1
NGC4274
Moment 0 ...
Moment 1 ...
Cube ...
NGC4274 1
NGC4062
Moment 0 ...
Moment 1 ...
Cube .

ID,Vrec,Velo Third,Colour
str7,float64,int64,str6
NGC0672,401879.99439243996,1,Blue
NGC4244,240079.99610904,1,Blue
NGC4448,376000.0,1,Blue
NGC0891,535196.68017412,2,Green
NGC2541,551879.99611629,2,Green
NGC5585,233899.9962807,1,Blue
NGC0949,629220.0,2,Green
NGC5229,369479.99668124,1,Blue
NGC0925,551879.99439244,2,Green


## HIX

In [20]:
mom1_list = glob.glob('/data/Surveys/HIX/Cubes/*.fits')
hix_tab = get_sorting(mom1_list)
hix_tab.write('HIX.csv', overwrite=True)
sort_the_data('/data/Surveys/HIX/', hix_tab)
hix_tab

8
7
8
NGC3001
Moment 0 ...
Moment 1 ...
Cube ...
NGC3001 1
ESO240-11
Moment 0 ...
Moment 1 ...
Cube ...
ESO240-11 1
IC4857
Moment 0 ...
Moment 1 ...
Cube ...
IC4857 1
ESO121-26
Moment 0 ...
Moment 1 ...
Cube ...
ESO121-26 1
ESO417-18
Moment 0 ...
Moment 1 ...
Cube ...
ESO417-18 1
ESO381-5
Moment 0 ...
Moment 1 ...
Cube ...
ESO381-5 1
ESO245-10
Moment 0 ...
Moment 1 ...
Cube ...
ESO245-10 1
NGC5161
Moment 0 ...
Moment 1 ...
Cube ...
NGC5161 1
ESO243-2
Moment 0 ...
Moment 1 ...
Cube ...
ESO243-2 1
NGC685
Moment 0 ...
Moment 1 ...
Cube ...
NGC685 1
ESO287-13
Moment 0 ...
Moment 1 ...
Cube ...
ESO287-13 1
ESO208-26
Moment 0 ...
Moment 1 ...
Cube ...
ESO208-26 1
ESO55-13
Moment 0 ...
Moment 1 ...
Cube ...
ESO55-13 1
ESO383-5
Moment 0 ...
Moment 1 ...
Cube ...
ESO383-5 1
ESO378-3
Moment 0 ...
Moment 1 ...
Cube ...
ESO378-3 1
NGC289
Moment 0 ...
Moment 1 ...
Cube ...
NGC289 1
ESO461-10
Moment 0 ...
Moment 1 ...
Cube ...
ESO461-10 1
ESO75-6
Moment 0 ...
Moment 1 ...
Cube ...
ESO75-6 1
NGC3261


ID,Vrec,Velo Third,Colour
str9,float64,int64,str6
NGC3001,2696000.0283019897,1,Blue
ESO240-11,2888999.9238432497,2,Green
IC4857,4720999.89894521,2,Green
ESO121-26,2283249.954024845,1,Blue
ESO417-18,4802999.9970589,2,Green
ESO381-5,5801000.101643715,3,Red
ESO245-10,5832000.05797139,3,Red
NGC5161,2295999.9925816553,1,Blue
ESO243-2,8939000.055782385,3,Red


## LVHIS

In [29]:
mom1_list = glob.glob('/data/Surveys/LVHIS/Cubes/*.fits')
lvhis_tab = get_sorting(mom1_list)
lvhis_tab.write('LVHIS.csv', overwrite=True)
sort_the_data('/data/Surveys/LVHIS/', lvhis_tab)
lvhis_tab

26
24
29
CEN06
Moment 0 ...
Moment 1 ...
Cube ...
CEN06 1
ESO410-5
Moment 0 ...
Moment 1 ...
Cube ...
ESO410-5 1
ESO245-7
Moment 0 ...
Moment 1 ...
Cube ...
ESO245-7 1
AM0605-341
Moment 0 ...
Moment 1 ...
Cube ...
AM0605-341 1
ESO174-1
Moment 0 ...
Moment 1 ...
Cube ...
ESO174-1 1
NGC2915
Moment 0 ...
Moment 1 ...
Cube ...
NGC2915 1
ESO383-87
Moment 0 ...
Moment 1 ...
Cube ...
ESO383-87 1
ESO137-18
Moment 0 ...
Moment 1 ...
Cube ...
ESO137-18 1
AM1321-304
Moment 0 ...
Moment 1 ...
Cube ...
AM1321-304 1
ESO115-21
Moment 0 ...
Moment 1 ...
Cube ...
ESO115-21 1
NGC55
Moment 0 ...
Moment 1 ...
Cube ...
NGC55 1
ESO149-3
Moment 0 ...
Moment 1 ...
Cube ...
ESO149-3 1
LVHIS054
Moment 0 ...
Moment 1 ...
Cube ...
LVHIS054 1
LVHIS032
Moment 0 ...
Moment 1 ...
Cube ...
LVHIS032 1
ESO349-31
Moment 0 ...
Moment 1 ...
Cube ...
ESO349-31 1
LVHIS061
Moment 0 ...
Moment 1 ...
Cube ...
LVHIS061 1
UGCA442
Moment 0 ...
Moment 1 ...
Cube ...
UGCA442 1
ESO223-9
Moment 0 ...
Moment 1 ...
Cube ...
ESO223-9 1
E

ID,Vrec,Velo Third,Colour
str11,float64,int64,str6
CEN06,608000.0,3,Red
ESO410-5,36000.0,1,Blue
ESO245-7,-32000.0,1,Blue
AM0605-341,768000.0,3,Red
ESO174-1,678000.0,3,Red
NGC2915,478000.0,2,Green
ESO383-87,328000.0,1,Blue
ESO137-18,598000.0,3,Red
AM1321-304,478000.0,2,Green


## SAURON

In [22]:
mom1_list = glob.glob('/data/Surveys/SAURON/Cubes/*.fits')
sauron_tab = get_sorting(mom1_list)
sauron_tab.write('SAURON.csv', overwrite=True)
sort_the_data('/data/Surveys/SAURON/', sauron_tab)
sauron_tab

3
3
4
NGC5308
Moment 0 ...
Moment 1 ...
Cube ...
NGC5308 1
NGC2549
Moment 0 ...
Moment 1 ...
Cube ...
NGC2549 1
NGC3414
Moment 0 ...
Moment 1 ...
Cube ...
NGC3414 1
NGC4150
Moment 0 ...
Moment 1 ...
Cube ...
NGC4150 1
NGC1023
Moment 0 ...
Moment 1 ...
Cube ...
NGC1023 1
NGC5198
Moment 0 ...
Moment 1 ...
Cube ...
NGC5198 1
NGC4278
Moment 0 ...
Moment 1 ...
Cube ...
NGC4278 1
NGC5982
Moment 0 ...
Moment 1 ...
Cube ...
NGC5982 1
NGC2768
Moment 0 ...
Moment 1 ...
Cube ...
NGC2768 1
NGC7457
Moment 0 ...
Moment 1 ...
Cube ...
NGC7457 1


ID,Vrec,Velo Third,Colour
str7,float64,int64,str6
NGC5308,800703.3356390449,3,Red
NGC2549,798492.862132255,3,Red
NGC3414,797281.34473319,2,Green
NGC4150,439996.75176467,1,Blue
NGC1023,656289.2736543,1,Blue
NGC5198,1131210.6293742,3,Red
NGC4278,701471.6621628099,2,Green
NGC5982,1610773.6859084899,3,Red
NGC2768,795178.038936165,2,Green


## SHREG

In [23]:
mom1_list = glob.glob('/data/Surveys/SHREG/Cubes/*.fits')
shreg_tab = get_sorting(mom1_list)
shreg_tab.write('SHREG.csv', overwrite=True)
sort_the_data('/data/Surveys/SHREG/', shreg_tab)
shreg_tab

16
14
23
ESO201-22
Moment 0 ...
Moment 1 ...
Cube ...
ESO201-22 1
ESO39-2
Moment 0 ...
Moment 1 ...
Cube ...
ESO39-2 1
NGC6920
Moment 0 ...
Moment 1 ...
Cube ...
NGC6920 1
NGC7098
Moment 0 ...
Moment 1 ...
Cube ...
NGC7098 1
NGC4930
Moment 0 ...
Moment 1 ...
Cube ...
NGC4930 1
ESO240-11
Moment 0 ...
Moment 1 ...
Cube ...
ESO240-11 1
ESO121-6
Moment 0 ...
Moment 1 ...
Cube ...
ESO121-6 1
ESO249-36
Moment 0 ...
Moment 1 ...
Cube ...
ESO249-36 1
ESO249-31
Moment 0 ...
Moment 1 ...
Cube ...
ESO249-31 1
ESO235-58
Moment 0 ...
Moment 1 ...
Cube ...
ESO235-58 1
ESO145-25
Moment 0 ...
Moment 1 ...
Cube ...
ESO145-25 1
ESO79-14
Moment 0 ...
Moment 1 ...
Cube ...
ESO79-14 1
NGC6221
Moment 0 ...
Moment 1 ...
Cube ...
NGC6221 1
ESO115-21
Moment 0 ...
Moment 1 ...
Cube ...
ESO115-21 1
NGC986
Moment 0 ...
Moment 1 ...
Cube ...
NGC986 1
ESO138-14
Moment 0 ...
Moment 1 ...
Cube ...
ESO138-14 1
ESO358-63
Moment 0 ...
Moment 1 ...
Cube ...
ESO358-63 1
NGC6850
Moment 0 ...
Moment 1 ...
Cube ...
NGC6850 1

ID,Vrec,Velo Third,Colour
str9,float64,int64,str6
ESO201-22,4102245.1074186997,3,Red
ESO39-2,2206815.6250011,3,Red
NGC6920,3050811.96614472,3,Red
NGC7098,2206815.6250011,3,Red
NGC4930,2206815.6250011,3,Red
ESO240-11,2840016.697590925,3,Red
ESO121-6,1151743.15836557,1,Blue
ESO249-36,730402.811380635,1,Blue
ESO249-31,941221.5020748349,1,Blue


## THINGS

In [24]:
mom1_list = glob.glob('/data/Surveys/THINGS/Cubes/*.fits')
things_tab = get_sorting(mom1_list)
things_tab.write('THINGS.csv', overwrite=True)
sort_the_data('/data/Surveys/THINGS/', things_tab)
things_tab

11
11
11
NGC5457
Moment 0 ...
Moment 1 ...
Cube ...
NGC5457 1
NGC3627
Moment 0 ...
Moment 1 ...
Cube ...
NGC3627 1
NGC3184
Moment 0 ...
Moment 1 ...
Cube ...
NGC3184 1
NGC1569
Moment 0 ...
Moment 1 ...
Cube ...
NGC1569 1
NGC3351
Moment 0 ...
Moment 1 ...
Cube ...
NGC3351 1
NGC4214
Moment 0 ...
Moment 1 ...
Cube ...
NGC4214 1
NGC5194
Moment 0 ...
Moment 1 ...
Cube ...
NGC5194 1
NGC628
Moment 0 ...
Moment 1 ...
Cube ...
NGC628 1
NGC3031
Moment 0 ...
Moment 1 ...
Cube ...
NGC3031 1
NGC6946
Moment 0 ...
Moment 1 ...
Cube ...
NGC6946 1
NGC4826
Moment 0 ...
Moment 1 ...
Cube ...
NGC4826 1
NGC2841
Moment 0 ...
Moment 1 ...
Cube ...
NGC2841 1
NGC3621
Moment 0 ...
Moment 1 ...
Cube ...
NGC3621 1
NGC2366
Moment 0 ...
Moment 1 ...
Cube ...
NGC2366 1
M81-DWA
Moment 0 ...
Moment 1 ...
Cube ...
M81-DWA 1
HOII
Moment 0 ...
Moment 1 ...
Cube ...
HOII 1
M81-DWB
Moment 0 ...
Moment 1 ...
Cube ...
M81-DWB 1
NGC7793
Moment 0 ...
Moment 1 ...
Cube ...
NGC7793 1
NGC4736
Moment 0 ...
Moment 1 ...
Cube ...
NG

ID,Vrec,Velo Third,Colour
str7,float64,int64,str6
NGC5457,178292.259772,2,Green
NGC3627,613470.0681195001,3,Red
NGC3184,500179.8220115,3,Red
NGC1569,-185267.186266,1,Blue
NGC3351,715131.1240105,3,Red
NGC4214,225822.3278635,2,Green
NGC5194,390517.869124,2,Green
NGC628,581601.4083390001,3,Red
NGC3031,-188078.7170825,1,Blue


## VIVA

In [25]:
mom1_list = glob.glob('/data/Surveys/VIVA/Cubes/*.fits')
viva_tab = get_sorting(mom1_list)
viva_tab.write('VIVA.csv', overwrite=True)
sort_the_data('/data/Surveys/VIVA/', viva_tab)
viva_tab

14
14
15
NGC4651
Moment 0 ...
Moment 1 ...
Cube ...
NGC4651 1
NGC4424
Moment 0 ...
Moment 1 ...
Cube ...
NGC4424 1
NGC4351
Moment 0 ...
Moment 1 ...
Cube ...
NGC4351 1
NGC4419
Moment 0 ...
Moment 1 ...
Cube ...
NGC4419 1
NGC4536
Moment 0 ...
Moment 1 ...
Cube ...
NGC4536 1
NGC4457
Moment 0 ...
Moment 1 ...
Cube ...
NGC4457 1
NGC4293
Moment 0 ...
Moment 1 ...
Cube ...
NGC4293 1
NGC4216
Moment 0 ...
Moment 1 ...
Cube ...
NGC4216 1
NGC4569
Moment 0 ...
Moment 1 ...
Cube ...
NGC4569 1
NGC4548
Moment 0 ...
Moment 1 ...
Cube ...
NGC4548 1
NGC4808
Moment 0 ...
Moment 1 ...
Cube ...
NGC4808 1
NGC4254
Moment 0 ...
Moment 1 ...
Cube ...
NGC4254 1
VCC2062
Moment 0 ...
Moment 1 ...
Cube ...
VCC2062 1
NGC4532
Moment 0 ...
Moment 1 ...
Cube ...
NGC4532 1
NGC4561
Moment 0 ...
Moment 1 ...
Cube ...
NGC4561 1
NGC4405
Moment 0 ...
Moment 1 ...
Cube ...
NGC4405 1
NGC4321
Moment 0 ...
Moment 1 ...
Cube ...
NGC4321 1
NGC4535
Moment 0 ...
Moment 1 ...
Cube ...
NGC4535 1
NGC4522
Moment 0 ...
Moment 1 ...
Cub

ID,Vrec,Velo Third,Colour
str7,float64,int64,str6
NGC4651,1418155118.315,3,Red
NGC4424,1419773013.37164,3,Red
NGC4351,1411095985.945,1,Blue
NGC4419,1421729978.235,3,Red
NGC4536,1413491324.485,2,Green
NGC4457,1417630856.155,3,Red
NGC4293,1417587243.735,3,Red
NGC4216,1421461927.215,3,Red
NGC4569,1422983817.205,3,Red


## VLA-ANGST

In [26]:
mom1_list = glob.glob('/data/Surveys/VLA-ANGST/Cubes/*.fits')
vla_angst_tab = get_sorting(mom1_list)
vla_angst_tab.write('VLA_ANGST.csv', overwrite=True)
sort_the_data('/data/Surveys/VLA-ANGST/', vla_angst_tab)
vla_angst_tab

9
9
10
NGC3741
Moment 0 ...
Moment 1 ...
Cube ...
NGC3741 1
MCG9-20-131
Moment 0 ...
Moment 1 ...
Cube ...
MCG9-20-131 1
DDO187
Moment 0 ...
Moment 1 ...
Cube ...
DDO187 1
KKH86
Moment 0 ...
Moment 1 ...
Cube ...
KKH86 1
DDO190
Moment 0 ...
Moment 1 ...
Cube ...
DDO190 1
DDO6
Moment 0 ...
Moment 1 ...
Cube ...
DDO6 1
UGC8833
Moment 0 ...
Moment 1 ...
Cube ...
UGC8833 1
KK230
Moment 0 ...
Moment 1 ...
Cube ...
KK230 1
GR8
Moment 0 ...
Moment 1 ...
Cube ...
GR8 1
KDG73
Moment 0 ...
Moment 1 ...
Cube ...
KDG73 1
UGC8508
Moment 0 ...
Moment 1 ...
Cube ...
UGC8508 1
ANTLIA
Moment 0 ...
Moment 1 ...
Cube ...
ANTLIA 1
NGC247
Moment 0 ...
Moment 1 ...
Cube ...
NGC247 1
DDO181
Moment 0 ...
Moment 1 ...
Cube ...
DDO181 1
UGC4483
Moment 0 ...
Moment 1 ...
Cube ...
UGC4483 1
NGC3109
Moment 0 ...
Moment 1 ...
Cube ...
NGC3109 1
DDO183
Moment 0 ...
Moment 1 ...
Cube ...
DDO183 1
SextansA
Moment 0 ...
Moment 1 ...
Cube ...
SextansA 1
DDO82
Moment 0 ...
Moment 1 ...
Cube ...
DDO82 1
BK3N
Moment 0 ...


ID,Vrec,Velo Third,Colour
str11,float64,int64,str6
NGC3741,302541.334621,3,Red
MCG9-20-131,209290.28101500002,2,Green
DDO187,208736.35105599998,1,Blue
KKH86,303457.62591485004,3,Red
DDO190,225768.5155105,2,Green
DDO6,314846.54037635,3,Red
UGC8833,290800.25355,2,Green
KK230,84554.2956555,1,Blue
GR8,247220.98593199998,2,Green


## WHISP

In [27]:
mom1_list = glob.glob('/data/Surveys/WHISP/Cubes/*.fits')
whisp_tab = get_sorting(mom1_list)
whisp_tab.write('WHISP.csv', overwrite=True)
sort_the_data('/data/Surveys/WHISP/', whisp_tab)
whisp_tab

55
55
56
UGC5717
Moment 0 ...
Moment 1 ...
Cube ...
UGC5717 1
UGC5997
Moment 0 ...
Moment 1 ...
Cube ...
UGC5997 1
UGC1541
Moment 0 ...
Moment 1 ...
Cube ...
UGC1541 1
UGC89
Moment 0 ...
Moment 1 ...
Cube ...
UGC89 1
UGC2023
Moment 0 ...
Filename: /data/Surveys/WHISP/Mom0/UGC2023_mom0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      85   (512, 512)   float32   
None
Moment 1 ...
Filename: /data/Surveys/WHISP/Mom1/UGC2023_mom1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      85   (512, 512)   float32   
None
Cube ...
Filename: /data/Surveys/WHISP/Cubes/UGC2023_cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      87   (512, 512, 127)   int16 (rescales to float32)   
None
UGC2023 0
UGC3407
Moment 0 ...
Moment 1 ...
Cube ...
UGC3407 1
UGC5452
Moment 0 ...
Moment 1 ...
Cube ...
UGC5452 1
UGC3354
Moment 0 ...
Moment 1 ...
Cube .

None
UGC1256 0
UGC3734
Moment 0 ...
Moment 1 ...
Cube ...
UGC3734 1
UGC5589
Moment 0 ...
Moment 1 ...
Cube ...
UGC5589 1
UGC4305
Moment 0 ...
Filename: /data/Surveys/WHISP/Mom0/UGC4305_mom0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      97   (512, 512)   float32   
None
Moment 1 ...
Filename: /data/Surveys/WHISP/Mom1/UGC4305_mom1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      97   (512, 512)   float32   
None
Cube ...
Filename: /data/Surveys/WHISP/Cubes/UGC4305_cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      99   (512, 512, 127)   int16 (rescales to float32)   
None
UGC4305 0
UGC7090
Moment 0 ...
Moment 1 ...
Cube ...
UGC7090 1
UGC3546
Moment 0 ...
Moment 1 ...
Cube ...
UGC3546 1
UGC2503
Moment 0 ...
Moment 1 ...
Cube ...
UGC2503 1
UGC731
Moment 0 ...
Filename: /data/Surveys/WHISP/Mom0/UGC731_mom0.fits
No.    Name

None
Moment 1 ...
Filename: /data/Surveys/WHISP/Mom1/UGC5253_mom1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      94   (512, 512)   float32   
None
Cube ...
Filename: /data/Surveys/WHISP/Cubes/UGC5253_cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      96   (512, 512, 63)   int16 (rescales to float32)   
None
UGC5253 0
UGC4605
Moment 0 ...
Moment 1 ...
Cube ...
UGC4605 1
UGC508
Moment 0 ...
Moment 1 ...
Cube ...
UGC508 1
UGC5960
Moment 0 ...
Moment 1 ...
Cube ...
UGC5960 1
UGC6921
Moment 0 ...
Moment 1 ...
Cube ...
UGC6921 1
UGC3642
Moment 0 ...
Moment 1 ...
Cube ...
UGC3642 1
UGC5721
Moment 0 ...
Moment 1 ...
Cube ...
UGC5721 1
UGC1281
Moment 0 ...
Filename: /data/Surveys/WHISP/Mom0/UGC1281_mom0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      97   (512, 512)   float32   
None
Moment 1 ...
Filename: /data/Surveys/WHISP/

None
Moment 1 ...
Filename: /data/Surveys/WHISP/Mom1/UGC5829_mom1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     104   (512, 512)   float32   
None
Cube ...
Filename: /data/Surveys/WHISP/Cubes/UGC5829_cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     106   (512, 512, 127)   int16 (rescales to float32)   
None
UGC5829 0
UGC6713
Moment 0 ...
Moment 1 ...
Cube ...
UGC6713 1
UGC528
Moment 0 ...
Filename: /data/Surveys/WHISP/Mom0/UGC528_mom0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      97   (512, 512)   float32   
None
Moment 1 ...
Filename: /data/Surveys/WHISP/Mom1/UGC528_mom1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      97   (512, 512)   float32   
None
Cube ...
Filename: /data/Surveys/WHISP/Cubes/UGC528_cube.fits
No.    Name      Ver    Type      Cards   Dimen

ID,Vrec,Velo Third,Colour
str7,float64,int64,str6
UGC5717,1411213981.529,2,Green
UGC5997,1413147094.162,2,Green
UGC1541,1391768021.96,1,Blue
UGC89,1396703104.724,1,Blue
UGC2023,1417056888.078,3,Red
UGC3407,1401297663.3702536,1,Blue
UGC5452,1412723433.568,2,Green
UGC3354,1403515573.089,1,Blue
UGC6944,1403813829.088,1,Blue


## SHIELD

In [28]:
mom1_list = glob.glob('/data/Surveys/SHIELD/Cubes/*.fits')
shield_tab = get_sorting(mom1_list)
shield_tab.write('SHIELD.csv', overwrite=True)
sort_the_data('/data/Surveys/SHIELD/', shield_tab)
shield_tab

4
4
4
AGC112521
Moment 0 ...
Moment 1 ...
Cube ...
AGC112521 1
AGC748778
Moment 0 ...
Moment 1 ...
Cube ...
AGC748778 1
AGC111946
Moment 0 ...
Moment 1 ...
Cube ...
AGC111946 1
AGC174585
Moment 0 ...
Moment 1 ...
Cube ...
AGC174585 1
AGC111164
Moment 0 ...
Moment 1 ...
Cube ...
AGC111164 1
AGC174605
Moment 0 ...
Moment 1 ...
Cube ...
AGC174605 1
AGC182595
Moment 0 ...
Moment 1 ...
Cube ...
AGC182595 1
AGC110482
Moment 0 ...
Moment 1 ...
Cube ...
AGC110482 1
AGC749241
Moment 0 ...
Moment 1 ...
Cube ...
AGC749241 1
AGC731457
Moment 0 ...
Moment 1 ...
Cube ...
AGC731457 1
AGC749237
Moment 0 ...
Moment 1 ...
Cube ...
AGC749237 1
AGC111977
Moment 0 ...
Moment 1 ...
Cube ...
AGC111977 1


ID,Vrec,Velo Third,Colour
str9,float64,int64,str6
AGC112521,273673.79871585226,1,Blue
AGC748778,257671.83838119314,1,Blue
AGC111946,366673.978206761,2,Green
AGC174585,355676.47501342755,2,Green
AGC111164,162674.15517008537,1,Blue
AGC174605,350671.83747019316,2,Green
AGC182595,397668.9842104452,3,Red
AGC110482,356673.79905985226,2,Green
AGC749241,450668.09237542376,3,Red
